In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
import os
os.environ["OPENAI_API_KEY"]= OPENAI_API_KEY

In [ ]:
!pip install -q langchain
!pip install -q ragas
!pip install -q faiss-cpu
!pip install -q langchain-openai
!pip install -q tiktoken

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

In [ ]:
## Load the Data
loader = TextLoader("/content/state_of_the_union.txt")
document = loader.load()

In [ ]:
## Chunking the Data

text_splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap= 50)
chunks = text_splitter.split_documents(document)

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS

vectordb = FAISS.from_documents(chunks, embeddings)


In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
# Defining LLM

llm = ChatOpenAI(temperature= 0.6)

In [ ]:
## Define Prompt Template

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
## Setting UP RAG Pipeline

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("What did president say about Justic Breyer?")

'The President thanked Justice Breyer for his service and mentioned nominating Judge Ketanji Brown Jackson to continue his legacy of excellence on the Supreme Court.'

In [ ]:
questions = ["what did the President say about Justic Breyer?",
           "What did the President say abou Intel's CEO?",
           "What did the President say about gun violence?"]

In [ ]:
ground_truth = ["The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service.",
              "The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion.",
              "The president asked Congress to pass proven measures to reduce gun violence."]

In [ ]:
answer = []
content = []

In [ ]:
# Inference

for query in questions:
  answer.append(chain.invoke(query))
  content.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

<ipython-input-26-aef51345ba2c>:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  content.append([docs.page_content for docs in retriever.get_relevant_documents(query)])


In [ ]:
## To dict
data = {
    "question": questions,
    "ground_truth": ground_truth,
    "answer": answer,
    "contexts": content
}

In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict(data)

In [ ]:
dataset

Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 3
})

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [ ]:
result = evaluate(dataset= dataset,
                  metrics = [
                      context_precision,
                      context_recall,
                      faithfulness,
                      answer_relevancy],
                  llm = llm,
                  embeddings =embeddings
                  )

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
result

{'context_precision': 0.9444, 'context_recall': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.9111}

In [ ]:
df = result.to_pandas()

In [ ]:
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,what did the President say about Justic Breyer?,"[Tonight, I’d like to honor someone who has de...",The President honored Justice Stephen Breyer f...,The president said that Justice Breyer has ded...,1.000000,1.0,1.0,0.885023
1,What did the President say abou Intel's CEO?,[Up to eight state-of-the-art factories in one...,"The President mentioned that Intel's CEO, Pat ...",The president said that Pat Gelsinger is ready...,1.000000,1.0,1.0,0.940327
2,What did the President say about gun violence?,[And I ask Congress to pass proven measures to...,The President called for Congress to pass prov...,The president asked Congress to pass proven me...,0.833333,1.0,1.0,0.907931
